In [1]:
import numpy as np
import copy
from collections import deque
import random

I realise, I could just implement a fucking deque to do all this. It'll be way simpler and should handle all the errors I'm getting.

Sigh.

In [2]:
class LSystemBoardGen:
    def __init__(self, size, agent_count) -> None:
        self.size = size
        self.agent_count = agent_count
        self.board = self.initialise_starting_board(self.size, self.agent_count)
        self.agents = self.assign_agent_states()

    def initialise_starting_board(self, size:int, agent_count):
        """Initialise as starting board of shape (size, size) with agent_count agents."""
        # :TO UPDATE
        board = np.zeros(shape=(size,size), dtype=int)
        for agent in range(1, 1+agent_count):
            tries = 0
            while(1):
                tries += 1
                if tries == 100:
                    print(f'ERROR - NO BOARD SPACES FOUND AFTER {100} TRIES')
                    break

                idx0 = np.random.randint(0, self.size)
                idx1 = np.random.randint(0, self.size)
                if board[idx0, idx1] != 0.0:
                    continue
                else:
                    board[idx0, idx1] = agent
                    break
        return board

    def assign_agent_states(self):
        """Construct agent from board info. 
        
        Each agent has the following attributes:
        (agent_num, deque)
        """
        agents = []
        for agent_num in range(1, 1+self.agent_count):
            pos = np.argwhere(self.board==agent_num)
            agent_deque = deque(pos)
            agents.append([agent_num, agent_deque])
        return agents

    def find_next_pos(self, loc:np.ndarray):
        """Find the next empty spot for an agent head/tail."""
        empty_places = np.argwhere(self.board==0)
        empty_place_distances = np.sqrt(np.sum((loc-empty_places)**2, axis=1))
        empty_neighbour_places = np.argwhere(empty_place_distances<1+1/9)
        empty_neighbour_coords = np.take(empty_places, empty_neighbour_places, axis=0)
        return empty_neighbour_coords[:,0,:]

    def push(self, agent_num):
        agent = self.agents[agent_num-1]
        growth_choice = np.random.randint(-1, 1) # choose to grow from the head or the tail, :TO UPDATE
        nbs = self.find_next_pos(agent[1][growth_choice])

        if len(nbs) == 0:
            return None
        else:
            growth_loc = np.random.randint(0, len(nbs)) # choose a random growth direction
            self.board[tuple(nbs[growth_loc])] = agent[0] # grow in a chosen direction

            # update deque to account for new growth
            if growth_choice == 0:
                agent[1].appendleft(nbs[growth_loc])
            elif growth_choice == -1:
                agent[1].append(nbs[growth_loc])

    def pull(self, agent_num):
        agent = self.agents[agent_num-1]
        shrink_choice = np.random.randint(-1, 1) #:TO UPDATE

        if len(agent[1]) == 1:
            return None
        
        else:
            self.board[tuple(agent[1][shrink_choice])] = 0
            if shrink_choice == 0:
                _ = agent[1].popleft()
            elif shrink_choice == -1:
                _ = agent[1].pop()

    def fill(self, n_steps:int=5, pushpullnone_ratios=[2, 1, 1]): # this is done IN-PLACE!!!
        # to generate a board, we'll need some parameters:
        ## push/pull strategy
        ## steps allowed in the push/pull
        ## simultaneous vs. global worm movement strategies
        for _ in range(n_steps):
            for agent in range(1, 1+len(self.agents)):
                action = random.choices(['push', 'pull', 'none'], weights=pushpullnone_ratios)[0]
                if action == 'push':
                    self.push(agent)
                elif action == 'pull':
                    self.pull(agent)
        return None

    def convert_to_jumanji(self, route=True):
        """Converts the generated board into a Jumanji-compatible one."""
        jumanji_board = copy.deepcopy(self.board)
        
        for agent in self.agents:
            # it _needs_ to be checked that the wires are at least 2 long for this procedure
            jumanji_board[tuple(agent[1][0])] = self.board[tuple(agent[1][0])]*3 + 1
            jumanji_board[tuple(agent[1][-1])] = self.board[tuple(agent[1][-1])]*3
            agent_body_length = len(agent[1]) - 2
            if agent_body_length > 0:
                for i in range(1, 1+agent_body_length):
                    if route:
                        jumanji_board[tuple(agent[1][i])] = self.board[tuple(agent[1][i])]*3-1
                    else:
                        jumanji_board[tuple(agent[1][i])] = 0
            
        return jumanji_board

    def return_training_board(self):
        return self.convert_to_jumanji(route=False)


In [46]:
Board = LSystemBoardGen(8, 5)
# print(Board.board, end='\n\n')

Board.fill(20, [2, 0.25, 1])
# print(Board.board, end='\n\n')
print(Board.convert_to_jumanji(route=False), '\n\n')
print(Board.convert_to_jumanji(route=True))

[[ 0  0  0  0  0  0  3  0]
 [ 0  0  0  0  0  0  0  7]
 [ 0  0  0  4  0  0  0  0]
 [ 0  0  0  0  0  6  0  0]
 [ 0  0 16  0  0  9  0  0]
 [ 0 15  0  0  0  0  0  0]
 [ 0  0 10 12  0  0  0  0]
 [ 0  0  0  0  0  0  0 13]] 


[[14 14 14  2  2  2  3  0]
 [14 14 14  2  2  2  5  7]
 [ 0 14 14  4  2  2  5  5]
 [14 14 14  2  2  6  5  5]
 [14  0 16  8  8  9  5  5]
 [14 15  8  8  8  8 11 11]
 [ 0  0 10 12 11 11 11 11]
 [ 0  0  0  0 11 11  0 13]]
